##### Copyright 2020 The TensorFlow IO Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Preparação e ampliação de dados de áudio

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/io/tutorials/audio"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/io/tutorials/audio.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/io/tutorials/audio.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
      <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/io/tutorials/audio.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Visão geral

Um dos maiores desafios no Reconhecimento de Fala Automático é a preparação e ampliação dos dados de áudio. A análise dos dados de áudio pode ser feita no domínio de tempo ou frequência, o que traz uma complexidade adicional em comparação a outras fontes de dados, como imagens.

Como parte do ecossistema do TensorFlow, o pacote `tensorflow-io` conta com algumas APIs relacionadas a áudio bastante úteis, que ajudam a preparar e ampliar os dados de áudio.

## Configuração

### Instale os pacotes necessários e reinicie o runtime

In [ ]:
!pip install tensorflow-io

## Uso

### Leia um arquivo de áudio

No TensorFlow IO, a classe `tfio.audio.AudioIOTensor` permite ler um arquivo de áudio em um `IOTensor` carregado de forma lazy (postergada):

In [ ]:
import tensorflow as tf
import tensorflow_io as tfio

audio = tfio.audio.AudioIOTensor('gs://cloud-samples-tests/speech/brooklyn.flac')

print(audio)

No exemplo acima, o arquivo `brooklyn.flac` vem de um clipe de áudio acessível publicamente no [Google Cloud](https://cloud.google.com/speech-to-text/docs/quickstart-gcloud).

O endereço `gs://cloud-samples-tests/speech/brooklyn.flac` do GCS é usado diretamente porque o GSC é um sistema de arquivo compatível com o TensorFlow. Além do formato `Flac`, os formatos `WAV`, `Ogg`, `MP3` e `MP4A` também são compatíveis com `AudioIOTensor` com detecção automática do formato de arquivo.

`AudioIOTensor` é carregado de maneira lazy, então somente o formato, o dtype e a taxa de amostragem são exibidos inicialmente. O formato do `AudioIOTensor` é representado como `[samples, channels]` (amostras, canais), ou seja, o clipe de áudio carregado tem um canal, com `28979` amostras `int16`.

O conteúdo do clipe de áudio só será lido conforme necessário, seja convertendo `AudioIOTensor` em um `Tensor` por meio de `to_tensor()`, seja recortando. O recorte é especialmente útil quando somente uma pequena parte de um clipe de áudio grande é necessário.

In [ ]:
audio_slice = audio[100:]

# remove last dimension
audio_tensor = tf.squeeze(audio_slice, axis=[-1])

print(audio_tensor)

O áudio pode ser reproduzido da seguinte forma:

In [ ]:
from IPython.display import Audio

Audio(audio_tensor.numpy(), rate=audio.rate.numpy())

É mais conveniente converter o tensor em números de ponto flutuante e exibir o clipe de áudio em um grafo:

In [ ]:
import matplotlib.pyplot as plt


tensor = tf.cast(audio_tensor, tf.float32) / 32768.0

plt.figure()
plt.plot(tensor.numpy())

### Retire o ruído

Às vezes, pode fazer sentido retirar o ruído do áudio, o que pode ser feito usando a API `tfio.audio.trim`. A API retorna um par de posição `[start, stop]` (início, fim) do segmento:

In [ ]:
position = tfio.audio.trim(tensor, axis=0, epsilon=0.1)
print(position)

start = position[0]
stop = position[1]
print(start, stop)

processed = tensor[start:stop]

plt.figure()
plt.plot(processed.numpy())

### Desvanecimento de início e fim

Uma técnica de engenharia de áudio muito útil é o desvanecimento, que aumenta ou diminui gradualmente os sinais de áudio. Isso pode ser feito usando `tfio.audio.fade`. `tfio.audio.fade` conta com diferentes formatos de desvanecimento, como `linear`, `logarithmic` (logarítmico) ou `exponential` (exponencial):

In [ ]:
fade = tfio.audio.fade(
    processed, fade_in=1000, fade_out=2000, mode="logarithmic")

plt.figure()
plt.plot(fade.numpy())

### Espectrograma

O processamento avançado de áudio costuma funcionar com alterações de frequência ao longo do tempo. No `tensorflow-io`, uma forma de onda pode ser convertida em um espectrograma por meio de `tfio.audio.spectrogram`:

In [ ]:
# Convert to spectrogram
spectrogram = tfio.audio.spectrogram(
    fade, nfft=512, window=512, stride=256)

plt.figure()
plt.imshow(tf.math.log(spectrogram).numpy())

Também é possível transformar em diferentes escalas:

In [ ]:
# Convert to mel-spectrogram
mel_spectrogram = tfio.audio.melscale(
    spectrogram, rate=16000, mels=128, fmin=0, fmax=8000)


plt.figure()
plt.imshow(tf.math.log(mel_spectrogram).numpy())

# Convert to db scale mel-spectrogram
dbscale_mel_spectrogram = tfio.audio.dbscale(
    mel_spectrogram, top_db=80)

plt.figure()
plt.imshow(dbscale_mel_spectrogram.numpy())

### SpecAugment

Além das APIs de preparação e ampliação de dados mencionadas acima, o pacote `tensorflow-io` também conta com ampliações avançadas de espectrograma, principalmente Mascaramento de Frequência e Mascaramento de Tempo discutidas em [SpecAugment: Um método simples de ampliação de dados para Reconhecimento de Fala Automático (Park et al., 2019)](https://arxiv.org/pdf/1904.08779.pdf).

#### Mascaramento de Frequência

No mascaramento de frequência, os canais de frequência `[f0, f0 + f)` são mascarados, em que `f` é escolhido em uma distribuição uniforme de `0` até o parâmetro de máscara de frequência `F`, e `f0` é escolhido no intervalo `(0, ν − f)` em que `ν` é o número de canais de frequência.

In [ ]:
# Freq masking
freq_mask = tfio.audio.freq_mask(dbscale_mel_spectrogram, param=10)

plt.figure()
plt.imshow(freq_mask.numpy())

#### Mascaramento de Tempo

No mascaramento de tempo, `t` passos de tempo consecutivos `[t0, t0 + t)` são mascarados, em que `t` é escolhido em uma distribuição uniforme de `0` até o parâmetro de máscara de tempo `T`, e `t0` é escolhido no intervalo `[0, τ − t)`, em que `τ` são os passos de tempo.

In [ ]:
# Time masking
time_mask = tfio.audio.time_mask(dbscale_mel_spectrogram, param=10)

plt.figure()
plt.imshow(time_mask.numpy())